<a href="https://colab.research.google.com/github/sagunkayastha/ILab_Tutorials/blob/master/NLP_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embedding

**Downloading and extracting word embedding**

In [0]:
import numpy as np

In [2]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove.42B.300d.zip

--2019-09-27 04:16:45--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2019-09-27 04:16:45--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2019-09-27 04:16:45--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [0]:
words, word_to_vec_map = read_glove_vecs('glove.42B.300d.txt')

In [0]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
king = word_to_vec_map["king"]
queen = word_to_vec_map["queen"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]


In [0]:
# Visualize the embedding
print(father)

**Cosine Similarity**
$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

In [0]:
def cosine_similarity(u, v):
    
    distance = 0.0
    
    dot = np.dot(u, v)
    
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    
    cosine_similarity = dot / np.dot(norm_u, norm_v)

    
    return cosine_similarity


In [0]:
print(cosine_similarity(father,mother))
print(cosine_similarity(king,queen))
print(cosine_similarity(king,rome))

0.8171303880464806
0.7596175811305904
0.39602592048853846


**A=B then C=?**

In [0]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    
    # word embeddings v_a, v_b and v_c 
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in words:        
        
        if w in [word_a, word_b, word_c] :
            continue
        
        
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity((e_b - e_a), (word_to_vec_map[w] - e_c))
        
       
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        
        
    return best_word

In [0]:
complete_analogy('France','Paris','Germany',word_to_vec_map)

'burger'

# RNN

In [0]:
!wget https://raw.githubusercontent.com/sagunkayastha/ILab_Tutorials/master/ISEAR.csv

--2019-09-26 16:13:48--  https://raw.githubusercontent.com/sagunkayastha/ILab_Tutorials/master/ISEAR.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957909 (935K) [text/plain]
Saving to: ‘ISEAR.csv.2’

ISEAR.csv.2         100%[===================>] 935.46K  --.-KB/s    in 0.009s  

2019-09-26 16:13:48 (106 MB/s) - ‘ISEAR.csv.2’ saved [957909/957909]



In [0]:
# import
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import string
import nltk

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

# load and preprocess the dataset
df = pd.read_csv('./ISEAR.csv',names = ['Emotions','Sentence'], index_col=False)
df.describe()

# preprocessing the dataset

# clean the dataset
df.dropna(axis=0, inplace = True)
df.describe()
# replacing misspelled data
df['Emotions'].replace(to_replace = 'guit', value='guilt', inplace = True)

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def clean_text(text):
    sw = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from','in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
    text = text.lower()
    text = nltk.word_tokenize(text)
    stem = WordNetLemmatizer()
    text = [stem.lemmatize(w) for w in text if w not in sw and w.isalnum()]
    text = ' '.join(text)
    return text
df['Sentence'] = df['Sentence'].map(lambda x: clean_text(x))
df.drop(df[df['Sentence'].apply(len) == 0].index, inplace=True)

# check the dataset after cleaning
# print(df['Emotions'].value_counts())

# split the dataset
X, y = df['Sentence'], df['Emotions']

# transform y to one-hot-encoding
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
onehot_y = np_utils.to_categorical(encoded_y)

# total number of unique words
vocab = set()
total_vocab = [vocab.add(el) for s in X.values for el in s.split(' ')]

In [0]:
# embeddings_index = {}
# f = open('glove.42B.300d.txt', encoding='utf8')
# for line in f:
#     values = line.split()
#     word = ''.join(values[:-300])
#     coefs = np.asarray(values[-300:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

In [0]:
# Tokenize the sentences
VOCABULARY_SIZE = len(total_vocab)  # Select an Appropriate Vobabulary Size
PADDED_LENGTH = 90  # Select an Appropriate padded Length for text
tokenizer = Tokenizer(VOCABULARY_SIZE) 
tokenizer.fit_on_texts(X)

# text to number
def preprocessing(X):
    seq = tokenizer.texts_to_sequences(X)
    data = sequence.pad_sequences(seq, maxlen = PADDED_LENGTH)
    return data
data = preprocessing(X)


# split test set and train set
X_train, X_test, y_train, y_test = train_test_split(data, onehot_y, test_size = 0.1, random_state = 101, stratify = onehot_y)

# model
LEARNING_RATE = 1e-3
EPOCH = 80
EMBEDDING_SIZE = 300
SEED = 2019
DROPOUTRATE = 0.3

In [0]:
embeddings_index = word_to_vec_map
embedding_matrix = np.zeros((VOCABULARY_SIZE, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.optimizers import Adam
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D

modelrnn = Sequential()
modelrnn.add(Embedding(VOCABULARY_SIZE, EMBEDDING_SIZE, input_length = PADDED_LENGTH,weights=[embedding_matrix], trainable= 'false',
                   embeddings_regularizer=regularizers.l1(0.001)))
modelrnn.add(SpatialDropout1D(0.2))


modelrnn.add(LSTM(128, dropout=DROPOUTRATE, recurrent_dropout=DROPOUTRATE, return_sequences = True))
modelrnn.add(LSTM(64, dropout=DROPOUTRATE, recurrent_dropout=DROPOUTRATE))
modelrnn.add(Dense(64, activation='relu'))
modelrnn.add(Dropout(DROPOUTRATE, seed = SEED))
modelrnn.add(Dense(128, activation='relu'))
modelrnn.add(Dropout(DROPOUTRATE, seed = SEED))
modelrnn.add(Dense(7, activation='softmax'))

optim = optimizers.Adam(lr = LEARNING_RATE,
                       beta_1=0.9,
                       beta_2=0.999,
                       epsilon=1e-08,
                       decay=LEARNING_RATE/EPOCH)

modelrnn.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
modelrnn.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 90, 300)           22623600  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 90, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 90, 128)           219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         

In [0]:
history = modelrnn.fit(X_train,y_train, batch_size=16, validation_split=0.2, epochs = EPOCH)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5410 samples, validate on 1353 samples
Epoch 1/80
5410/5410 [==============================] - 132s 24ms/step - loss: 356.2661 - acc: 0.2362 - val_loss: 157.8668 - val_acc: 0.3341
Epoch 2/80
5410/5410 [==============================] - 129s 24ms/step - loss: 83.3340 - acc: 0.3299 - val_loss: 40.1806 - val_acc: 0.3865
Epoch 3/80
5410/5410 [==============================] - 129s 24ms/step - loss: 27.3153 - acc: 0.3743 - val_loss: 19.5375 - val_acc: 0.3851
Epoch 4/80
5410/5410 [==============================] - 130s 24ms/step - loss: 16.2753 - acc: 0.4083 - val_loss: 13.5264 - val_acc: 0.4671
Epoch 5/80
5410/5410 [==============================] - 129s 24ms/step - loss: 11.6380 - acc: 0.4335 - val_loss: 9.8552 - val_acc: 0.4568
Epoch 6/80
5410/5410 [==============================] - 129s 24ms/step - loss: 8.5374 - acc: 0.4671 - val_loss: 7.1794 - val_acc: 0.4982
Epoch 7/80
5410/5410 [===

# OpenAI

In [0]:
!git clone https://github.com/openai/gpt-2.git
%cd gpt-2/

Cloning into 'gpt-2'...
remote: Enumerating objects: 209, done.
remote: Total 209 (delta 0), reused 0 (delta 0), pack-reused 209
Receiving objects: 100% (209/209), 4.37 MiB | 15.49 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/gpt-2


In [0]:
!pip install -r requirements.txt

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 604kB 10.5MB/s 
     |████████████████████████████████| 51kB 19.2MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=e888a5bce565753582a2e3b8c77c18a77067f5a30317eb1706ac841e10c28b1a
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533196 sha256=54ca929d667b58cf6ace272057218c6f215021eea22e584969ce40bd081ed37d
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
!python download_model.py 124M

Fetching checkpoint: 1.00kit [00:00, 589kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 27.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 609kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.7Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.34Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 35.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 26.1Mit/s]                                                       


In [0]:
!python src/interactive_conditional_samples.py 124M


2019-09-26 07:51:24.735194: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-26 07:51:24.787428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-26 07:51:24.788252: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-09-26 07:51:24.794955: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-09-26 07:51:25.032229: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-09-26 07:51:25.137095: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.1